In [15]:
!pip install konlpy

# Word2Vec
- 구글에서 발표 (가장 많이 사용)
1. CBOW
- 주변 단어를 중심으로 타겟 단어 예측
2. skip-gram
- 타겟 단어를 중심으로 주변 단어 예측

In [5]:
# import
from gensim.models import Word2Vec
from konlpy.tag import Komoran
import urllib.request
import time

In [4]:
# 리뷰 다운로드
print("네이버 영화 리뷰 다운로드 시작")
url = "https://raw.githubusercontent.com/e9t/nsmc/master/ratings.txt"
urllib.request.urlretrieve(url, filename=url.split("/")[-1])

네이버 영화 리뷰 다운로드 시작


('ratings.txt', <http.client.HTTPMessage at 0x7f5df5ea5250>)

In [5]:
# 네이버 영화 리뷰 데이터 읽어옴
def read_review_data(filename):
    with open(filename, 'r') as f:
        data = [line.split('\t') for line in f.read().splitlines()]
        data = data[1:] # header 제거
    return data


# 측정 시작
start = time.time()

# 리뷰 파일 읽어오기
print('1) 말뭉치 데이터 읽기 시작')
review_data = read_review_data('./ratings.txt')
print(len(review_data)) # 리뷰 데이터 전체 개수
print('1) 말뭉치 데이터 읽기 완료: ', time.time() - start)

1) 말뭉치 데이터 읽기 시작
200000
1) 말뭉치 데이터 읽기 완료:  0.4902045726776123


In [6]:
# 문장단위로 명사만 추출해 학습 입력 데이터로 만듬
print('2) 형태소에서 명사만 추출 시작')
komoran = Komoran()
docs = [komoran.nouns(sentence[1]) for sentence in review_data]
print('2) 형태소에서 명사만 추출 완료: ', time.time() - start)

# word2vec 모델 학습
print('3) word2vec 모델 학습 시작')
model = Word2Vec(sentences=docs, size=200, window=4, min_count=2, sg=1)
print('3) word2vec 모델 학습 완료: ', time.time() - start)

# 모델 저장
print('4) 학습된 모델 저장 시작')
model.save('nvmc.model')
print('4) 학습된 모델 저장 완료: ', time.time() - start)

# 학습된 말뭉치 개수, 코퍼스 내 전체 단어 개수
print("corpus_count : ", model.corpus_count)
print("corpus_total_words : ", model.corpus_total_words)

2) 형태소에서 명사만 추출 시작
2) 형태소에서 명사만 추출 완료:  289.66458535194397
3) word2vec 모델 학습 시작
3) word2vec 모델 학습 완료:  315.3549828529358
4) 학습된 모델 저장 시작
4) 학습된 모델 저장 완료:  315.7180507183075
corpus_count :  200000
corpus_total_words :  1076896


In [9]:
from gensim.models import Word2Vec

# 모델 로딩
model = Word2Vec.load('nvmc.model')
print("corpus_total_words : ", model.corpus_total_words)

# '사랑'이란 단어로 생성한 단어 임베딩 벡터
print('사랑 : ', model.wv['사랑'])

# 단어 유사도 계산
print("일요일 = 월요일\t", model.wv.similarity(w1='일요일', w2='월요일'))
print("안성기 = 배우\t", model.wv.similarity(w1='안성기', w2='배우'))
print("대기업 = 삼성\t", model.wv.similarity(w1='대기업', w2='삼성'))
print("일요일 != 삼성\t", model.wv.similarity(w1='일요일', w2='삼성'))
print("히어로 != 삼성\t", model.wv.similarity(w1='히어로', w2='삼성'))

# 가장 유사한 단어 추출
print(model.wv.most_similar("안성기", topn=5))
print(model.wv.most_similar("시리즈", topn=5))


corpus_total_words :  1076896
사랑 :  [ 1.54961705e-01 -8.28421209e-03 -4.06622648e-01 -1.35005593e-01
  2.75653362e-01 -8.24820176e-02 -5.70251532e-02  2.17864692e-01
  5.06871620e-05  4.20566835e-02 -2.51299828e-01 -1.68055996e-01
  3.62588197e-01  1.30160570e-01  1.11356057e-01  1.01998411e-01
 -2.39766881e-01 -1.75024360e-01 -6.94807572e-03 -8.86054859e-02
 -3.62326950e-02  2.31147602e-01 -2.75029391e-01 -9.42782778e-03
 -3.10203195e-01  2.54819751e-01  5.46838017e-03 -5.76698743e-02
  1.15322292e-01 -3.74119550e-01  9.96087790e-02  3.65488172e-01
 -6.60605505e-02 -2.99641266e-02 -1.28484592e-01 -2.33996473e-02
  7.60770738e-02 -1.42497882e-01 -8.54229927e-02  3.53047222e-01
  4.05172966e-02  2.83779621e-01  5.68072796e-01  1.39622599e-01
  2.76407272e-01 -1.65754035e-01  4.68611985e-01  2.30613112e-01
 -4.20536995e-01 -1.57247961e-01 -1.88040257e-01 -2.05474004e-01
 -3.24679077e-01  1.14672847e-01 -7.01559857e-02 -2.86520272e-01
  2.00340584e-01  7.10725263e-02 -2.00896353e-01  3.45

# 텍스트 유사도



## n-gram 유사도
- 이웃 단어의 출현 빈도를 통계적으로 표현

In [13]:
from konlpy.tag import Komoran

In [1]:
# 어절 단위 n-gram
def word_ngram(bow, num_gram):
    text = tuple(bow)
    ngrams = [text[x:x + num_gram] for x in range(0, len(text))]
    return tuple(ngrams)

In [2]:
# 음절 n-gram 분석
def phoneme_ngram(bow, num_gram):
    sentence = ' '.join(bow)
    text = tuple(sentence)
    slen = len(text)
    ngrams = [text[x:x + num_gram] for x in range(0, slen)]
    return ngrams

In [11]:
# 유사도 계산
def similarity(doc1, doc2):
    cnt = 0
    for token in doc1:
        if token in doc2:
            cnt = cnt + 1

    return cnt/len(doc1)

In [12]:
# 2-gram 유사도
sentence1 = '6월에 뉴턴은 선생님의 제안으로 트리니티에 입학하였다'
sentence2 = '6월에 뉴턴은 선생님의 제안으로 대학교에 입학하였다'
sentence3 = '나는 맛잇는 밥을 뉴턴 선생님과 함께 먹었습니다.'

komoran = Komoran()
bow1 = komoran.nouns(sentence1)
bow2 = komoran.nouns(sentence2)
bow3 = komoran.nouns(sentence3)

doc1 = word_ngram(bow1, 2)
doc2 = word_ngram(bow2, 2)
doc3 = word_ngram(bow3, 2)

print("doc1 :",doc1)
print("doc2 :",doc2)
print("doc3 :",doc3)

r1 = similarity(doc1, doc2)
r2 = similarity(doc3, doc1)
print(r1)
print(r2)

doc1 : (('6월', '뉴턴'), ('뉴턴', '선생님'), ('선생님', '제안'), ('제안', '트리니티'), ('트리니티', '입학'), ('입학',))
doc2 : (('6월', '뉴턴'), ('뉴턴', '선생님'), ('선생님', '제안'), ('제안', '대학교'), ('대학교', '입학'), ('입학',))
doc3 : (('맛', '밥'), ('밥', '뉴턴'), ('뉴턴', '선생'), ('선생', '님과 함께'), ('님과 함께',))
0.6666666666666666
0.0


## 코사인 유사도
- 벡터의 크기가 중요하지 않을때 사용
- 다양한 차원에서 적용가능. 실무에서 많이 사용

In [ ]:
from konlpy.tag import Komoran
import numpy as np
from numpy import dot
from numpy.linalg import norm

In [ ]:
# 코사인 유사도 계산
def cos_sim(vec1, vec2):
    return dot(vec1, vec2) / (norm(vec1) * norm(vec2))

In [ ]:
# TDM 만들기
def make_term_doc_mat(sentence_bow, word_dics):
    freq_mat = {}

    for word in word_dics:
        freq_mat[word] = 0

    for word in word_dics:
        if word in sentence_bow:
            freq_mat[word] += 1

    return freq_mat

# 회귀
Dense(1) ->마지막  
model.compile(optimizer = 'adam', loss='mse')

# 이항분류 (0,1)
Dense(1, activation = 'sigmoid') -> 마지막  
model.compile(optimizeer = 'adam', loss='binary_crossentropy', metrics=['acc'])

# 다중분류
Dense(10, activation = 'softmax') -> 마지막 (10 : 분류항 개수)  
model.compile(optimizer = 'adam', loss='sparse_categorical_crossentropy', metrics=['acc'])

# 다중분류 (희소벡터 = 원핫인코딩 반환)
model.compile(optimizer = 'adam', loss='categorical_crossentropy', metrics=['acc'])

# 합성곱 (convolution)
- 특징을 추출
- 원하는 패턴이 얼마나 있나 계산

# 풀링
- 메모리의 크기를 줄이기 위해 사용하는것 => 풀링
- 커널을 안줄어들게 하기위해 패딩 사용


# 챗봇 문답 감정 분류 모델

In [ ]:
!pip install tensorflow

In [20]:
# 필요한 모듈 임포트
import pandas as pd
import tensorflow as tf
from tensorflow.keras import preprocessing
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Dense, Dropout, Conv1D, GlobalMaxPool1D, concatenate

In [27]:
# 데이터 읽어오기
url = "https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv"
data = pd.read_csv(url, delimiter=',')
features = data['Q'].tolist()
labels = data['label'].tolist()

# 단어 인덱스 시퀀스 벡터
corpus = [preprocessing.text.text_to_word_sequence(text) for text in features]

In [28]:
tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(corpus)
sequences = tokenizer.texts_to_sequences(corpus)
word_index = tokenizer.word_index
MAX_SEQ_LEN = 15  # 단어 시퀀스 벡터 크기
padded_seqs = preprocessing.sequence.pad_sequences(sequences, maxlen=MAX_SEQ_LEN, padding='post')

In [31]:
# 학습용, 검증용, 테스트용 데이터셋 생성 ➌
# 학습셋:검증셋:테스트셋 = 7:2:1
ds = tf.data.Dataset.from_tensor_slices((padded_seqs, labels))
ds = ds.shuffle(len(features))
train_size = int(len(padded_seqs) * 0.7)
val_size = int(len(padded_seqs) * 0.2)
test_size = int(len(padded_seqs) * 0.1)
train_ds = ds.take(train_size).batch(20)
val_ds = ds.skip(train_size).take(val_size).batch(20)
test_ds = ds.skip(train_size + val_size).take(test_size).batch(20)

In [33]:
# 하이퍼파라미터 설정
# 함수형 코드
dropout_prob = 0.5
EMB_SIZE = 128
EPOCH = 5
VOCAB_SIZE = len(word_index) + 1  # 전체 단어 수

# CNN 모델 정의
input_layer = Input(shape=(MAX_SEQ_LEN,))
embedding_layer = Embedding(VOCAB_SIZE, EMB_SIZE, input_length=MAX_SEQ_LEN)(input_layer)
dropout_emb = Dropout(rate=dropout_prob)(embedding_layer)

conv1 = Conv1D(filters=128, kernel_size=3, padding='valid', activation=tf.nn.relu)(dropout_emb)
pool1 = GlobalMaxPool1D()(conv1)
conv2 = Conv1D(filters=128, kernel_size=4, padding='valid', activation=tf.nn.relu)(dropout_emb)
pool2 = GlobalMaxPool1D()(conv2)
conv3 = Conv1D(filters=128, kernel_size=5, padding='valid', activation=tf.nn.relu)(dropout_emb)
pool3 = GlobalMaxPool1D()(conv3)

In [34]:
# 3, 4, 5- gram 이후 합치기
concat = concatenate([pool1, pool2, pool3])
hidden = Dense(128, activation=tf.nn.relu)(concat)
dropout_hidden = Dropout(rate=dropout_prob)(hidden)
logits = Dense(3, name='logits')(dropout_hidden)
predictions = Dense(3, activation=tf.nn.softmax)(logits)

# 모델 생성
model = Model(inputs=input_layer, outputs=predictions)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 모델 학습
model.fit(train_ds, validation_data=val_ds, epochs=EPOCH, verbose=1)

# 모델 평가(테스트 데이터셋 이용)
loss, accuracy = model.evaluate(test_ds, verbose=1)
print('Accuracy: %f' % (accuracy * 100))
print('loss: %f' % (loss))

# 모델 저장
model.save('cnn_model.h5')

Epoch 1/5
414/414 [==============================] - 18s 13ms/step - loss: 0.9035 - accuracy: 0.5644 - val_loss: 0.6793 - val_accuracy: 0.7039
Epoch 2/5
414/414 [==============================] - 4s 11ms/step - loss: 0.5733 - accuracy: 0.7815 - val_loss: 0.2920 - val_accuracy: 0.9006
Epoch 3/5
414/414 [==============================] - 4s 11ms/step - loss: 0.3305 - accuracy: 0.8881 - val_loss: 0.1699 - val_accuracy: 0.9425
Epoch 4/5
414/414 [==============================] - 4s 11ms/step - loss: 0.2135 - accuracy: 0.9328 - val_loss: 0.0992 - val_accuracy: 0.9657
Epoch 5/5
60/60 [==============================] - 0s 5ms/step - loss: 0.0637 - accuracy: 0.9797
Accuracy: 97.969544
loss: 0.063654


In [35]:
import tensorflow as tf
import pandas as pd
from tensorflow.keras.models import Model, load_model
from tensorflow.keras import preprocessing

# 데이터 읽어오기
url = "https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv"
data = pd.read_csv(url, delimiter=',')
features = data['Q'].tolist()
labels = data['label'].tolist()

# 단어 인덱스 시퀀스 벡터
corpus = [preprocessing.text.text_to_word_sequence(text) for text in features]
tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(corpus)
sequences = tokenizer.texts_to_sequences(corpus)
MAX_SEQ_LEN = 15 # 단어 시퀀스 벡터 크기
padded_seqs = preprocessing.sequence.pad_sequences(sequences, maxlen=MAX_SEQ_LEN, padding='post')

In [37]:
# 테스트용 데이터셋 생성
ds = tf.data.Dataset.from_tensor_slices((padded_seqs, labels))
ds = ds.shuffle(len(features))
test_ds = ds.take(2000).batch(20) # 테스트 데이터셋

# 감정 분류 CNN 모델 불러오기
model = load_model('cnn_model.h5')
model.summary()
model.evaluate(test_ds, verbose=2)

# 테스트용 데이터셋의 10212번째 데이터 출력
print("단어 시퀀스 : ", corpus[10212])
print("단어 인덱스 시퀀스 : ", padded_seqs[10212])
print("문장 분류(정답) : ", labels[10212])

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 15)]         0           []                               
                                                                                                  
 embedding_1 (Embedding)        (None, 15, 128)      1715072     ['input_2[0][0]']                
                                                                                                  
 dropout_1 (Dropout)            (None, 15, 128)      0           ['embedding_1[0][0]']            
                                                                                                  
 conv1d_3 (Conv1D)              (None, 13, 128)      49280       ['dropout_1[0][0]']              
                                                                                              

In [38]:
# 테스트용 데이터셋의 10212번째 데이터 감정 예측
picks = [10212]
predict = model.predict(padded_seqs[picks])
predict_class = tf.math.argmax(predict, axis=1)
print("감정 예측 점수 : ", predict)
print("감정 예측 클래스 : ", predict_class.numpy())

감정 예측 점수 :  [[1.2372795e-05 1.5071144e-05 9.9997258e-01]]
감정 예측 클래스 :  [2]
